#### Importa os pacotes

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

#### Siglas dos Times

In [3]:
de_para_siglas = pd.read_csv('de_para_siglas.csv').set_index("sigla").to_dict()['nome']

#### Separa as informações no padrão do nome do arquivo 

In [254]:
def parse_filename(file_name):
    fl_name = file_name.replace(".csv", "").split("_")
    
    away_team = str(fl_name[0])
    home_team = str(fl_name[2])
    
    date_game = datetime.strptime(fl_name[3] + fl_name[4] + fl_name[5], '%b%d%Y')
    
    stat_team = fl_name[6]
    
    for key in de_para_siglas.keys():
        away_team_full = away_team.replace(key, de_para_siglas[key])
        stat_team_full = stat_team.replace(key, de_para_siglas[key])
        home_team_full = home_team.replace(key, de_para_siglas[key])
    
    df = pd.DataFrame(columns=["home_team", "away_team",
                               "date", "team"])
    df.loc[0] = [home_team, away_team,
                 date_game, stat_team]
    
    df["home_full"] = df["home_team"].replace(de_para_siglas, regex=True)
    df["away_full"] = df["away_team"].replace(de_para_siglas, regex=True)
    df["team_full"] = df["team"].replace(de_para_siglas, regex=True)
    
    return(df)

#### Lê o dataframe e retorna sem a coluna extra

In [255]:
def load_df(file_name):
    df = pd.read_csv("NBA_Games/dfs/" + file_name)
    df = df.drop(["Unnamed: 0"], axis=1)
    return(df)

#### Junta as estatísticas básicas com as avançadas

In [262]:
def join_basic_adv_df(file_name_prefix):
    df_adv = load_df(file_name_prefix + "_Advanced.csv")
    df_basic = load_df(file_name_prefix + "_Basic.csv")
    
    df_full = pd.concat([df_basic, df_adv.drop(["Starters", "MP"], axis=1)], axis=1)
    columns_to_numeric = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       '+/-', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%',
       'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg']

    df_full["MP"] = df_full["MP"].str.replace(":", ".") 
    df_full[columns_to_numeric] = df_full[columns_to_numeric].apply(pd.to_numeric, errors='coerce')
    
    info = parse_filename(file_name)
    df_full[list(info.columns)] = pd.concat([info]*len(df_full), ignore_index=True)
    
    df_full = df_full[["Starters"]  + list(info.columns) + columns_to_numeric]
    
    return(df_full)

#### Teste para um jogo

In [263]:
file_name = "ATL_@_BKN_Apr_02_2017_ATL"
df_full = join_basic_adv_df(file_name)

In [264]:
df_full.head(5)

,Starters,home_team,away_team,date,team,home_full,away_full,team_full,MP,FG,...,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg
0,Dennis Schroder,BKN,ATL,2017-04-02,ATL,Brooklyn Nets,Atlanta Hawks,Atlanta Hawks,40.52,7.0,...,2.3,10.7,6.2,43.1,2.4,2.4,20.0,26.9,76.0,92.0
1,Taurean Waller-Prince,BKN,ATL,2017-04-02,ATL,Brooklyn Nets,Atlanta Hawks,Atlanta Hawks,36.08,5.0,...,2.6,12.1,7.0,11.4,2.7,2.7,0.0,17.3,98.0,90.0
2,Tim Hardaway,BKN,ATL,2017-04-02,ATL,Brooklyn Nets,Atlanta Hawks,Atlanta Hawks,35.51,5.0,...,0.0,18.3,8.5,5.7,0.0,0.0,29.8,20.6,79.0,97.0
3,Dwight Howard,BKN,ATL,2017-04-02,ATL,Brooklyn Nets,Atlanta Hawks,Atlanta Hawks,25.25,4.0,...,3.7,42.9,21.9,0.0,0.0,3.9,39.2,22.1,80.0,88.0
4,Ersan Ilyasova,BKN,ATL,2017-04-02,ATL,Brooklyn Nets,Atlanta Hawks,Atlanta Hawks,20.18,1.0,...,0.0,0.0,0.0,17.1,2.4,4.8,14.5,14.9,70.0,93.0


#### Carrega as lista de arquivos de todos os jogos

In [265]:
directory = "NBA_Games/dfs/"

files_list = []

for filename in os.listdir(directory):
    if filename.endswith(".csv"): 
        file_name = filename.replace("_Basic.csv", "")
        file_name = file_name.replace("_Advanced.csv", "")
        files_list.append(file_name)
        
files_list = list(dict.fromkeys(files_list))

#### Gera DataFrame completo com informações

In [274]:
df_resp = None

for file_name in files_list:
    if(df_resp is None):
        df_resp = join_basic_adv_df(file_name)
    else:
        df_resp = pd.concat([df_resp, join_basic_adv_df(file_name)], axis=0).reset_index(drop=True)

df_resp['year_month'] = df_resp['date'].map(lambda x: 100*x.year + x.month)
df_resp["game"] = str(df_resp['away_team'] + " @ " + df_resp['home_team'] + " " + df_resp["date"].map(lambda x: x.strftime('%Y-%m-%d'))

In [296]:
df_resp.to_csv("nba_games_dataset.csv")

In [316]:
df_resp.columns

Index(['Starters', 'home_team', 'away_team', 'date', 'team', 'home_full',
       'away_full', 'team_full', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PTS', '+/-', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg',
       'year_month', 'game'],
      dtype='object')

In [ ]:
# Basic Box Score Stats

# MP -- Minutes Played
# FG -- Field Goals
# FGA -- Field Goal Attempts
# FG% -- Field Goal Percentage
# 3P -- 3-Point Field Goals
# 3PA -- 3-Point Field Goal Attempts
# 3P% -- 3-Point Field Goal Percentage
# FT -- Free Throws
# FTA -- Free Throw Attempts
# FT% -- Free Throw Percentage
# ORB -- Offensive Rebounds
# DRB -- Defensive Rebounds
# TRB -- Total Rebounds
# AST -- Assists
# STL -- Steals
# BLK -- Blocks
# TOV -- Turnovers
# PF -- Personal Fouls
# PTS -- Points
# +/- -- Plus/Minus

# Advanced Box Score Stats
# MP -- Minutes Played
# TS% -- True Shooting Percentage
# A measure of shooting efficiency that takes into account 2-point field goals, 3-point field goals, and free throws.
# eFG% -- Effective Field Goal Percentage
# This statistic adjusts for the fact that a 3-point field goal is worth one more point than a 2-point field goal.
# 3PAr -- 3-Point Attempt Rate
# Percentage of FG Attempts from 3-Point Range
# FTr -- Free Throw Attempt Rate
# Number of FT Attempts Per FG Attempt
# ORB% -- Offensive Rebound Percentage
# An estimate of the percentage of available offensive rebounds a player grabbed while he was on the floor.
# DRB% -- Defensive Rebound Percentage
# An estimate of the percentage of available defensive rebounds a player grabbed while he was on the floor.
# TRB% -- Total Rebound Percentage
# An estimate of the percentage of available rebounds a player grabbed while he was on the floor.
# AST% -- Assist Percentage
# An estimate of the percentage of teammate field goals a player assisted while he was on the floor.
# STL% -- Steal Percentage
# An estimate of the percentage of opponent possessions that end with a steal by the player while he was on the floor.
# BLK% -- Block Percentage
# An estimate of the percentage of opponent two-point field goal attempts blocked by the player while he was on the floor.
# TOV% -- Turnover Percentage
# An estimate of turnovers committed per 100 plays.
# USG% -- Usage Percentage
# An estimate of the percentage of team plays used by a player while he was on the floor.
# ORtg -- Offensive Rating
# An estimate of points produced (players) or scored (teams) per 100 possessions
# DRtg -- Defensive Rating
# An estimate of points allowed per 100 possessions

In [ ]:
columns_sum = ['MP', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PTS', '+/-', '3PAr', 'FTr', 'ORtg', 'DRtg']
columns_avg = ['FG%', '3P%', 'FT%', 'TS%', 'eFG%', 'ORB%', 'DRB%',
              'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
              ]

In [325]:
game_totals = df_resp[df_resp.Starters == "Team Totals"].groupby(["game", "team"], as_index=False).sum()

In [23]:
game_totals.to_csv("game_totals.csv")

In [4]:
game_totals = pd.read_csv("game_totals.csv")
game_totals = game_totals.drop(["Unnamed: 0"], axis=1)

In [22]:
game_totals.head()

,game,team,MP,FG,FGA,FG%,3P,3PA,3P%,FT,...,STL%,BLK%,TOV%,USG%,ORtg,DRtg,year_month,home,away,fl_home
0,ATL @ BKN 2015-11-17,ATL,240.0,33.0,76.0,0.434,12.0,29.0,0.414,10.0,...,7.6,10.9,19.6,100.0,96.1,98.3,201511,BKN,ATL,0
1,ATL @ BKN 2015-11-17,BKN,240.0,37.0,84.0,0.440,9.0,20.0,0.450,7.0,...,15.3,6.4,10.9,100.0,98.3,96.1,201511,BKN,ATL,1
2,ATL @ BKN 2017-01-10,ATL,240.0,44.0,92.0,0.478,7.0,23.0,0.304,22.0,...,10.9,20.7,9.9,100.0,116.0,96.2,201701,BKN,ATL,0
3,ATL @ BKN 2017-01-10,BKN,240.0,35.0,87.0,0.402,10.0,29.0,0.345,17.0,...,5.0,5.8,14.9,100.0,96.2,116.0,201701,BKN,ATL,1
4,ATL @ BKN 2017-04-02,ATL,240.0,30.0,80.0,0.375,7.0,24.0,0.292,15.0,...,9.0,8.2,17.4,100.0,82.4,91.4,201704,BKN,ATL,0


In [21]:
game_totals["fl_home"] = np.where(game_totals["game"].str[6:9] == game_totals["team"], 1, 0)

In [7]:
game_totals.columns

Index(['game', 'team', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', '+/-', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%',
       'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'year_month'],
      dtype='object')

In [26]:
home_games = game_totals[game_totals["fl_home"] == 1].set_index("game")
away_games = game_totals[game_totals["fl_home"] == 0].set_index("game")

In [29]:
all_games = home_games.join(away_games, how="inner", lsuffix="_home", rsuffix="_away")
all_games.drop(["home_home", "home_away", "year_month_home", "year_month_away", "fl_home_away",
                "fl_home_home", "away_home", "away_away", "MP_away"], axis=1, inplace=True)

In [56]:
all_games.columns

Index(['team_home', 'MP_home', 'FG_home', 'FGA_home', 'FG%_home', '3P_home',
       '3PA_home', '3P%_home', 'FT_home', 'FTA_home', 'FT%_home', 'ORB_home',
       'DRB_home', 'TRB_home', 'AST_home', 'STL_home', 'BLK_home', 'TOV_home',
       'PF_home', 'PTS_home', '+/-_home', 'TS%_home', 'eFG%_home', '3PAr_home',
       'FTr_home', 'ORB%_home', 'DRB%_home', 'TRB%_home', 'AST%_home',
       'STL%_home', 'BLK%_home', 'TOV%_home', 'USG%_home', 'ORtg_home',
       'DRtg_home', 'team_away', 'FG_away', 'FGA_away', 'FG%_away', '3P_away',
       '3PA_away', '3P%_away', 'FT_away', 'FTA_away', 'FT%_away', 'ORB_away',
       'DRB_away', 'TRB_away', 'AST_away', 'STL_away', 'BLK_away', 'TOV_away',
       'PF_away', 'PTS_away', '+/-_away', 'TS%_away', 'eFG%_away', '3PAr_away',
       'FTr_away', 'ORB%_away', 'DRB%_away', 'TRB%_away', 'AST%_away',
       'STL%_away', 'BLK%_away', 'TOV%_away', 'USG%_away', 'ORtg_away',
       'DRtg_away', 'date'],
      dtype='object')

In [61]:
all_games.head()

,team_home,MP_home,FG_home,FGA_home,FG%_home,3P_home,3PA_home,3P%_home,FT_home,FTA_home,...,TRB%_away,AST%_away,STL%_away,BLK%_away,TOV%_away,USG%_away,ORtg_away,DRtg_away,date,team_home_game_num
game,,,,,,,,,,,,,,,,,,,,,
DET @ ATL 2015-10-27,ATL,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,...,59.6,62.2,5.2,5.5,12.3,100.0,111.2,98.6,2015-10-27,1
NOP @ GSW 2015-10-27,GSW,240.0,41.0,96.0,0.427,9.0,30.0,0.300,20.0,22.0,...,37.1,60.0,9.0,4.5,15.9,100.0,94.9,110.9,2015-10-27,1
CLE @ CHI 2015-10-27,CHI,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,...,51.5,68.4,5.0,10.3,9.0,100.0,95.5,97.5,2015-10-27,1
DAL @ PHX 2015-10-28,PHX,240.0,34.0,87.0,0.391,6.0,24.0,0.250,21.0,33.0,...,47.5,60.0,6.9,4.8,7.5,100.0,108.9,93.2,2015-10-28,1
DEN @ HOU 2015-10-28,HOU,240.0,30.0,87.0,0.345,8.0,35.0,0.229,17.0,26.0,...,52.7,65.0,9.3,19.2,19.5,100.0,108.2,87.6,2015-10-28,1


In [99]:
def get_season(date):
    ano = date.year
    if(date.month >= 10):
        return ano + 1
    return ano

In [100]:
get_season(all_games["date"][0])

2016

In [101]:
all_games["date"] = [datetime.strptime(str(x)[10:20], '%Y-%m-%d') for x in all_games.index]
all_games["season"] = [get_season(x) for x in all_games.date]
all_games = all_games.sort_values('date')

In [102]:
all_games["team_home_game_num"] = all_games.groupby(['team_home', 'season']).cumcount() + 1
all_games["team_away_game_num"] = all_games.groupby(['team_away', 'season']).cumcount() + 1

In [103]:
all_games.head()

,team_home,MP_home,FG_home,FGA_home,FG%_home,3P_home,3PA_home,3P%_home,FT_home,FTA_home,...,STL%_away,BLK%_away,TOV%_away,USG%_away,ORtg_away,DRtg_away,date,team_home_game_num,team_away_game_num,season
game,,,,,,,,,,,,,,,,,,,,,
DET @ ATL 2015-10-27,ATL,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,...,5.2,5.5,12.3,100.0,111.2,98.6,2015-10-27,1,1,2016
NOP @ GSW 2015-10-27,GSW,240.0,41.0,96.0,0.427,9.0,30.0,0.300,20.0,22.0,...,9.0,4.5,15.9,100.0,94.9,110.9,2015-10-27,1,1,2016
CLE @ CHI 2015-10-27,CHI,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,...,5.0,10.3,9.0,100.0,95.5,97.5,2015-10-27,1,1,2016
MIN @ LAL 2015-10-28,LAL,240.0,35.0,90.0,0.389,9.0,34.0,0.265,32.0,35.0,...,3.0,7.1,7.5,100.0,111.9,110.9,2015-10-28,1,1,2016
DAL @ PHX 2015-10-28,PHX,240.0,34.0,87.0,0.391,6.0,24.0,0.250,21.0,33.0,...,6.9,4.8,7.5,100.0,108.9,93.2,2015-10-28,1,1,2016


In [104]:
all_games[all_games["team_home"] == "ATL"]["PTS_home"]

game
DET @ ATL 2015-10-27     94.0
CHA @ ATL 2015-10-30     97.0
BKN @ ATL 2015-11-04    101.0
WAS @ ATL 2015-11-07    114.0
MIN @ ATL 2015-11-09    107.0
NOP @ ATL 2015-11-11    106.0
UTA @ ATL 2015-11-15     96.0
SAC @ ATL 2015-11-18    103.0
BOS @ ATL 2015-11-24    121.0
OKC @ ATL 2015-11-30    106.0
TOR @ ATL 2015-12-02     86.0
LAL @ ATL 2015-12-04    100.0
SAS @ ATL 2015-12-12     78.0
MIA @ ATL 2015-12-14     88.0
PHI @ ATL 2015-12-16    127.0
POR @ ATL 2015-12-21    106.0
DET @ ATL 2015-12-23    107.0
NYK @ ATL 2015-12-26    117.0
NYK @ ATL 2016-01-05    101.0
CHI @ ATL 2016-01-09    120.0
BKN @ ATL 2016-01-16    114.0
ORL @ ATL 2016-01-18     98.0
LAC @ ATL 2016-01-27     83.0
MEM @ ATL 2016-03-12     95.0
IND @ ATL 2016-03-13    104.0
DEN @ ATL 2016-03-17    116.0
HOU @ ATL 2016-03-19    109.0
WAS @ ATL 2016-03-21    102.0
MIL @ ATL 2016-03-25    101.0
CLE @ ATL 2016-04-01    108.0
                        ...  
LAC @ ATL 2017-11-22    103.0
NYK @ ATL 2017-11-24    116.0
TOR @

In [84]:
teste_rolling = all_games[all_games["team_home"] == "ATL"].groupby('season')['PTS_home'].apply(lambda x:x.rolling(center=False,window=5).mean())

In [107]:
all_games[all_games["team_home"] == "ATL"].groupby('season').describe()

+/-_away                              +/-_home       ...   \
          count mean std min 25% 50% 75% max    count mean  ...    
season                                                      ...    
2016        0.0  NaN NaN NaN NaN NaN NaN NaN      0.0  NaN  ...    
2017        0.0  NaN NaN NaN NaN NaN NaN NaN      0.0  NaN  ...    
2018        0.0  NaN NaN NaN NaN NaN NaN NaN      0.0  NaN  ...    

       team_away_game_num       team_home_game_num                        \
                      75%   max              count  mean        std  min   
season                                                                     
2016                30.75  40.0               38.0  19.5  11.113055  1.0   
2017                29.00  41.0               40.0  20.5  11.690452  1.0   
2018                26.25  37.0               36.0  18.5  10.535654  1.0   

                                  
          25%   50%    75%   max  
season                            
2016    10.25  19.5  28.75  38.0  
2017    10.75  20.5  30.25  40.0  
2018     9.75  18.5  27.25  36.0  

[3 rows x 552 columns]

In [106]:
all_games[all_games["team_away"] == "ATL"].groupby('season').describe()

+/-_away                              +/-_home       ...   \
          count mean std min 25% 50% 75% max    count mean  ...    
season                                                      ...    
2016        0.0  NaN NaN NaN NaN NaN NaN NaN      0.0  NaN  ...    
2017        0.0  NaN NaN NaN NaN NaN NaN NaN      0.0  NaN  ...    
2018        0.0  NaN NaN NaN NaN NaN NaN NaN      0.0  NaN  ...    

       team_away_game_num       team_home_game_num                             \
                      75%   max              count       mean        std  min   
season                                                                          
2016                 32.5  43.0               43.0  20.279070  11.189548  1.0   
2017                 28.0  37.0               37.0  19.675676  11.267096  2.0   
2018                 26.5  35.0               35.0  17.628571  11.358039  1.0   

                                
         25%   50%   75%   max  
season                          
2016    11.0  21.0  29.5  38.0  
2017    10.0  19.0  31.0  37.0  
2018     7.0  17.0  29.5  35.0  

[3 rows x 552 columns]

In [82]:
all_games[all_games["team_home"] == "ATL"]["PTS_home"].rolling(5).mean()

game
DET @ ATL 2015-10-27      NaN
CHA @ ATL 2015-10-30      NaN
BKN @ ATL 2015-11-04      NaN
WAS @ ATL 2015-11-07      NaN
MIN @ ATL 2015-11-09    102.6
NOP @ ATL 2015-11-11    105.0
UTA @ ATL 2015-11-15    104.8
SAC @ ATL 2015-11-18    105.2
BOS @ ATL 2015-11-24    106.6
OKC @ ATL 2015-11-30    106.4
TOR @ ATL 2015-12-02    102.4
LAL @ ATL 2015-12-04    103.2
SAS @ ATL 2015-12-12     98.2
MIA @ ATL 2015-12-14     91.6
PHI @ ATL 2015-12-16     95.8
POR @ ATL 2015-12-21     99.8
DET @ ATL 2015-12-23    101.2
NYK @ ATL 2015-12-26    109.0
NYK @ ATL 2016-01-05    111.6
CHI @ ATL 2016-01-09    110.2
BKN @ ATL 2016-01-16    111.8
ORL @ ATL 2016-01-18    110.0
LAC @ ATL 2016-01-27    103.2
MEM @ ATL 2016-03-12    102.0
IND @ ATL 2016-03-13     98.8
DEN @ ATL 2016-03-17     99.2
HOU @ ATL 2016-03-19    101.4
WAS @ ATL 2016-03-21    105.2
MIL @ ATL 2016-03-25    106.4
CLE @ ATL 2016-04-01    107.2
                        ...  
LAC @ ATL 2017-11-22    107.8
NYK @ ATL 2017-11-24    110.2
TOR @